In [11]:
import random

from main import *

In [ ]:
client = DuckdbClient(use_hive_partitioning=True)
client.execute(
    """
    CREATE OR REPLACE SECRET secret (
         TYPE S3,
         PROVIDER CREDENTIAL_CHAIN
    );
    """
)

items = []
for collection in HLS_COLLECTIONS:
    items = client.search(
        href=HLS_STAC_GEOPARQUET_HREF.format(collection=collection),
        datetime="2025-10-01T00:00:00Z/2025-10-02T00:00:00Z",
    )


In [3]:
items[0]

{'type': 'Feature',
 'stac_version': '1.1.0',
 'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json',
  'https://stac-extensions.github.io/projection/v1.0.0/schema.json',
  'https://stac-extensions.github.io/view/v1.0.0/schema.json',
  'https://stac-extensions.github.io/scientific/v1.0.0/schema.json'],
 'id': 'HLS.S30.T14TMK.2025274T173141.v2.0',
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-100.183173, 40.644795],
     [-98.971849, 40.650853],
     [-99.325483, 39.661151],
     [-100.166132, 39.655753],
     [-100.183173, 40.644795]]]]},
 'bbox': [-100.183173, 39.655753, -98.971849, 40.650853],
 'links': [{'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-public/HLSS30.020/HLS.S30.T14TMK.2025274T173141.v2.0/HLS.S30.T14TMK.2025274T173141.v2.0_stac.json',
   'rel': 'self',
   'type': 'application/json'},
  {'href': 'https://doi.org/10.5067/HLS/HLSS30.002', 'rel': 'cite-as'}],
 'assets': {'B01': {'href': 'https://data.lpdaac.earthdatacloud

In [4]:
test = client.search(
    href=HLS_STAC_GEOPARQUET_HREF.format(collection="HLSL30_2.0"),
    datetime="2025-01-01T00:00:00Z/2025-12-31T23:59:59Z",
    filter={
        "op": "like",
        "args": [
            {
                "property": "id"
            },
            "%.T14TMK.%"
        ]
    }
)

len(test)

96

In [10]:
stack = odc.stac.load(
    [Item.from_dict(item) for item in test],
    stac_cfg=HLS_ODC_STAC_CONFIG,
    bands=["red", "green", "blue", "Fmask"],
    chunks={"x": 256, "y": 256},
    groupby=group_by_sensor_and_date,
).sortby("time")

stack

<xarray.Dataset> Size: 9GB
Dimensions:      (y: 3660, x: 3660, time: 95)
Coordinates:
  * y            (y) float64 29kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
  * x            (x) float64 29kB 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05
  * time         (time) datetime64[ns] 760B 2025-01-01T17:18:51.220000 ... 20...
    spatial_ref  int32 4B 32614
Data variables:
    red          (time, y, x) int16 3GB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    green        (time, y, x) int16 3GB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    blue         (time, y, x) int16 3GB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>
    Fmask        (time, y, x) uint8 1GB dask.array<chunksize=(1, 256, 256), meta=np.ndarray>

In [16]:
rand_x = random.sample(range(3600), 5)
rand_y = random.sample(range(3600), 5)

samp = stack.isel(x=rand_x, y=rand_y)
samp

<xarray.Dataset> Size: 17kB
Dimensions:      (y: 5, x: 5, time: 95)
Coordinates:
  * y            (y) float64 40B 4.463e+06 4.443e+06 ... 4.444e+06 4.455e+06
  * x            (x) float64 40B 4.155e+05 4.121e+05 ... 4.85e+05 4.163e+05
  * time         (time) datetime64[ns] 760B 2025-01-01T17:18:51.220000 ... 20...
    spatial_ref  int32 4B 32614
Data variables:
    red          (time, y, x) int16 5kB dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
    green        (time, y, x) int16 5kB dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
    blue         (time, y, x) int16 5kB dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
    Fmask        (time, y, x) uint8 2kB dask.array<chunksize=(1, 5, 5), meta=np.ndarray>

In [17]:
%%time
samp.load()

2025-11-20 12:46:08,692 - INFO - GDAL signalled an error: err_no=4, msg="`/vsicurl/https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T14TMK.2025217T171843.v2.0/HLS.L30.T14TMK.2025217T171843.v2.0.B02.tif' not recognized as being in a supported file format."
2025-11-20 12:46:08,693 - ERROR - Aborting load due to failure while reading: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T14TMK.2025217T171843.v2.0/HLS.L30.T14TMK.2025217T171843.v2.0.B02.tif:1


CPU times: user 3.17 s, sys: 771 ms, total: 3.94 s
Wall time: 24.8 s


RasterioIOError: '/vsicurl/https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/HLSL30.020/HLS.L30.T14TMK.2025217T171843.v2.0/HLS.L30.T14TMK.2025217T171843.v2.0.B02.tif' not recognized as being in a supported file format.